# Collaboration and Competition

---

Congratulations for completing the third project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893) program!  In this notebook, you will learn how to control agents in a more challenging environment, where the goal is to train a team of agents to play soccer.  **Note that this exercise is optional!**

### 1. Start the Environment

We begin by importing the necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [1]:
from unityagents import UnityEnvironment
import numpy as np

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Soccer.app"`
- **Windows** (x86): `"path/to/Soccer_Windows_x86/Soccer.exe"`
- **Windows** (x86_64): `"path/to/Soccer_Windows_x86_64/Soccer.exe"`
- **Linux** (x86): `"path/to/Soccer_Linux/Soccer.x86"`
- **Linux** (x86_64): `"path/to/Soccer_Linux/Soccer.x86_64"`
- **Linux** (x86, headless): `"path/to/Soccer_Linux_NoVis/Soccer.x86"`
- **Linux** (x86_64, headless): `"path/to/Soccer_Linux_NoVis/Soccer.x86_64"`

For instance, if you are using a Mac, then you downloaded `Soccer.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Soccer.app")
```

In [2]:
env = UnityEnvironment(file_name="./Soccer_Windows_x86_64/Soccer.exe") # Soccer

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 2
        Number of External Brains : 2
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: GoalieBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 112
        Number of stacked Vector Observation: 3
        Vector Action space type: discrete
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 
Unity brain name: StrikerBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 112
        Number of stacked Vector Observation: 3
        Vector Action space type: discrete
        Vector Action space size (per agent): 6
        Vector Action descriptions: , , , , , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we obtain separate brains for the striker and goalie agents.

In [3]:
# print the brain names
print(env.brain_names)

# set the goalie brain
g_brain_name = env.brain_names[0]
g_brain = env.brains[g_brain_name]

# set the striker brain
s_brain_name = env.brain_names[1]
s_brain = env.brains[s_brain_name]

['GoalieBrain', 'StrikerBrain']


### 2. Examine the State and Action Spaces

Run the code cell below to print some information about the environment.

In [4]:
# reset the environment
env_info = env.reset(train_mode=True)

# number of agents 
num_g_agents = len(env_info[g_brain_name].agents)
print('Number of goalie agents:', num_g_agents)
num_s_agents = len(env_info[s_brain_name].agents)
print('Number of striker agents:', num_s_agents)

# number of actions
g_action_size = g_brain.vector_action_space_size
print('Number of goalie actions:', g_action_size)
s_action_size = s_brain.vector_action_space_size
print('Number of striker actions:', s_action_size)

# examine the state space 
g_states = env_info[g_brain_name].vector_observations
g_state_size = g_states.shape[1]
print('There are {} goalie agents. Each receives a state with length: {}'.format(g_states.shape[0], g_state_size))
s_states = env_info[s_brain_name].vector_observations
s_state_size = s_states.shape[1]
print('There are {} striker agents. Each receives a state with length: {}'.format(s_states.shape[0], s_state_size))

Number of goalie agents: 2
Number of striker agents: 2
Number of goalie actions: 4
Number of striker actions: 6
There are 2 goalie agents. Each receives a state with length: 336
There are 2 striker agents. Each receives a state with length: 336


### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agents and receive feedback from the environment.

Once this cell is executed, you will watch the agents' performance, if they select actions at random with each time step.  A window should pop up that allows you to observe the agents.

Of course, as part of the project, you'll have to change the code so that the agents are able to use their experiences to gradually choose better actions when interacting with the environment!

In [ ]:
def RandomActionsAgents():
    for i in range(2):                                         # play game for 2 episodes
        env_info = env.reset(train_mode=False)                 # reset the environment    
        g_states = env_info[g_brain_name].vector_observations  # get initial state (goalies)
        s_states = env_info[s_brain_name].vector_observations  # get initial state (strikers)
        
        g_scores = np.zeros(num_g_agents)                      # initialize the score (goalies)
        s_scores = np.zeros(num_s_agents)                      # initialize the score (strikers)
        
        while True:
            # select actions and send to environment
            g_actions = np.random.randint(g_action_size, size=num_g_agents)
            s_actions = np.random.randint(s_action_size, size=num_s_agents)
            actions = dict(zip([g_brain_name, s_brain_name], 
                               [g_actions, s_actions]))
            env_info = env.step(actions)                       

            # get next states
            g_next_states = env_info[g_brain_name].vector_observations         
            s_next_states = env_info[s_brain_name].vector_observations

            # get reward and update scores
            g_rewards = env_info[g_brain_name].rewards  
            s_rewards = env_info[s_brain_name].rewards
            g_scores += g_rewards
            s_scores += s_rewards

            # check if episode finished
            done = np.any(env_info[g_brain_name].local_done)  

            # roll over states to next time step
            g_states = g_next_states
            s_states = s_next_states

            # exit loop if episode finished
            if done:                                           
                break
        print('Scores from episode {}: {} (goalies), {} (strikers)'.format(i+1, g_scores, s_scores))

import random
import torch
from unityagents import UnityEnvironment
import numpy as np
from collections import deque
from itertools import count
import time

import matplotlib.pyplot as plt
%matplotlib inline

from ddpg_agent import Agent
        
def DDPGMA(n_episodes=4000, print_every=100, target_score=0.5, max_t=1000):
    env_info = env.reset(train_mode=True)
    
    goaliesAgents = []
    strikersAgents = []
    
    g_states = env_info[g_brain_name].vector_observations  # get initial state (goalies)
    s_states = env_info[s_brain_name].vector_observations  # get initial state (strikers)
    
    #print(g_state_size)
    #print(s_state_size)
    
    g_scores_deque = deque(maxlen=100)
    g_scores = []
    
    s_scores_deque = deque(maxlen=100)
    s_scores = []
    
    g_agent = Agent(state_size=g_state_size, action_size=g_action_size, random_seed=429)    
    s_agent = Agent(state_size=s_state_size, action_size=s_action_size, random_seed=429)    
    
    
    for i_episode in range(0, n_episodes + 1):
        env_info = env.reset(train_mode=True)
        g_states = env_info[g_brain_name].vector_observations  # get initial state (goalies)
        s_states = env_info[s_brain_name].vector_observations  # get initial state (strikers)
 
        g_agent.reset()
        s_agent.reset()

        g_score = np.zeros(num_g_agents)
        s_score = np.zeros(num_s_agents)

        while True:
        #for t in range(max_t):
            g_actions = g_agent.act(g_states)
            s_actions = s_agent.act(s_states)

            g_actions_4step = g_actions
            s_actions_4step = s_actions


            g_actions = np.argmax(g_actions, axis=1)
            s_actions = np.argmax(s_actions, axis=1)

            g_actions = np.expand_dims(g_actions, axis=1)
            s_actions = np.expand_dims(s_actions, axis=1)

            actions = dict(zip([g_brain_name, s_brain_name], [g_actions, s_actions]))

            env_info = env.step(actions)

            g_next_state = env_info[g_brain_name].vector_observations
            s_next_state = env_info[s_brain_name].vector_observations

            g_rewards = env_info[g_brain_name].rewards  
            s_rewards = env_info[s_brain_name].rewards

            g_dones = env_info[g_brain_name].local_done
            s_dones = env_info[s_brain_name].local_done

            #print(type(s_states)) 
            #print("states s", s_states.shape)
            #print(type(s_actions)) 
            #print("actions s 4 step", s_actions_4step.shape)
            ##print(type(s_rewards))
            #print(type(s_next_state)) 
            #print("next state s", s_next_state.shape)
            ##print(type(s_dones))

            s_agent.step(s_states, s_actions_4step, s_rewards, s_next_state, s_dones)
            g_agent.step(g_states, g_actions_4step, g_rewards, g_next_state, g_dones)

            g_states = g_next_state
            s_states = s_next_state

            g_score += g_rewards
            s_score += s_rewards

            if np.any(g_dones) or np.any(s_dones):
                break

        g_scores_deque.append(np.mean(g_score))
        s_scores_deque.append(np.mean(s_score))

        g_scores.append(np.mean(g_score))
        s_scores.append(np.mean(s_score))

        if i_episode % 100 == 0:
            print('\rEpisode {}\tAverage Score - g:{:.2f} s:{:.2f}'.format(i_episode, np.mean(g_scores_deque), np.mean(s_scores_deque)))

            torch.save(g_agent.actor_local.state_dict(), 'checkpoint_actor_soccer_g.pth')
            torch.save(g_agent.critic_local.state_dict(), 'checkpoint_critic_soccer_g.pth')   

            torch.save(s_agent.actor_local.state_dict(), 'checkpoint_actor_soccer_s.pth')
            torch.save(s_agent.critic_local.state_dict(), 'checkpoint_critic_soccer_s.pth')   

    return g_scores, s_scores
            

g_scores, s_scores = DDPGMA()


fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(1, len(g_scores)+1), g_scores, 'r-')
plt.plot(np.arange(1, len(s_scores)+1), s_scores, 'b-')
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.show()

Episode 0	Average Score - g:0.15 s:-0.15
Episode 100	Average Score - g:0.08 s:-0.08
Episode 200	Average Score - g:0.20 s:-0.20
Episode 300	Average Score - g:-0.01 s:0.01
Episode 400	Average Score - g:0.15 s:-0.15
Episode 500	Average Score - g:0.04 s:-0.04
Episode 600	Average Score - g:0.08 s:-0.08
Episode 700	Average Score - g:-0.02 s:0.02
Episode 800	Average Score - g:0.14 s:-0.14
Episode 900	Average Score - g:0.07 s:-0.07
Episode 1000	Average Score - g:0.06 s:-0.06
Episode 1100	Average Score - g:0.14 s:-0.14
Episode 1200	Average Score - g:0.03 s:-0.03
Episode 1300	Average Score - g:0.05 s:-0.05
Episode 1400	Average Score - g:0.05 s:-0.05
Episode 1500	Average Score - g:0.08 s:-0.08


When finished, you can close the environment.

In [ ]:
env.close()

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```